In [1]:
import csv
from bs4 import BeautifulSoup

In [20]:
from selenium import webdriver


In [23]:
driver = webdriver.Chrome(executable_path = "C:\\webdriver\\chromedriver.exe")

C:\Users\Vivek Pinto\AppData\Local\Temp\ipykernel_16136\1156067254.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "C:\\webdriver\\chromedriver.exe")


In [24]:
url = "https://www.amazon.in"
driver.get(url)

In [27]:
def get_url(search_term):
    """generate a url from search term"""
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss'
    search_term = search_term.replace(' ','+')
    return template.format(search_term)


In [28]:
url = get_url('ultrawide monitor')
print(url)

https://www.amazon.in/s?k=ultrawide+monitor&ref=nb_sb_noss


In [29]:
driver.get(url)

## Extract the collection

In [30]:
soup =  BeautifulSoup(driver.page_source, 'html.parser')

In [31]:
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

In [32]:
len(results)

22

## Prototype the record

In [33]:
item = results[0]

In [34]:
atag = item.h2.a

In [35]:
description = atag.text.strip()

In [36]:
url = 'https://www.amazon.in' + atag.get('href')

In [37]:
price_parent = item.find('span', 'a-price') 

In [38]:
price = price_parent.find('span', 'a-offscreen').text

In [39]:
rating = item.i.text

In [44]:
review_count = item.find('span', {'class': 'a-size-base'}).text

## Generalize the pattern

In [45]:
def extract_record(item):
    """"Extract and return data from a single record"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    #price
    price_parent = item.find('span', 'a-price') 
    price = price_parent.find('span', 'a-offscreen').text
    
    #rank and rating
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base'}).text
    
    result = (description, price, rating, review_count, url)
    
    return result
    
    

In [46]:
records = []
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

for item in results:
    records.append(extract_record(item))

AttributeError: 'NoneType' object has no attribute 'text'

## Error handling

In [47]:
def extract_record(item):
    """"Extract and return data from a single record"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span', 'a-price') 
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try: 
        #rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result

In [48]:
records = []
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [49]:
records[0]

('Lenovo Q-Series 60.45 cm (24 inch) FHD IPS Ultraslim Monitor, 2x3W Speakers, 75Hz, AMD FreeSync, HDMI, Natural Low Blue Light, TUV Eyesafe® Vision Health Board Compliant, Lenovo Artery Control Q24i-1L',
 '₹14,489',
 '4.4 out of 5 stars',
 '1,236',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A01492841EWG1CDWC47CM&url=%2FLenovo-23-8-inch-Ultraslim-2xSpeakers-Q24i-1L%2Fdp%2FB095348ZMH%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dultrawide%2Bmonitor%26qid%3D1654676949%26sr%3D8-1-spons%26psc%3D1&qualifier=1654676949&id=2904229419326851&widgetName=sp_atf')

In [50]:
for row in records:
    print(row[1])

₹14,489
₹21,990
₹17,499
₹18,828
₹19,999
₹25,499
₹41,953
₹42,799
₹69,999
₹30,990
₹21,800
₹24,436
₹42,999
₹27,999
₹15,489
₹1,69,999
₹48,999
₹34,999
₹76,990
₹39,999
₹31,999
₹12,099


## Getting the next page

In [51]:
def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss'
    search_term = search_term.replace(' ','+')
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url
    

## Putting it all together

In [54]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss'
    search_term = search_term.replace(' ','+')
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

def extract_record(item):
    """"Extract and return data from a single record"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span', 'a-price') 
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try: 
        #rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    """Run main program routine"""
    #startup the webdriver
    driver = webdriver.Chrome(executable_path = "C:\\webdriver\\chromedriver.exe")
    
    record = []
    
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type' : 's-search-result'})
        
        for item in results: 
            record = extract_record(item)
            if record:
                records.append(record)
                
                
                
    driver.close()
    
    #save data to csv file
    with open ('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)
    


In [ ]:
main('ultrawide')

C:\Users\Vivek Pinto\AppData\Local\Temp\ipykernel_16136\381595495.py:47: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "C:\\webdriver\\chromedriver.exe")
